### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

### 安装特定依赖

In [ ]:
!cd modelscope-agent && pip install -r requirements.txt
! pip install transformers_stream_generator
! pip install invisible-watermark>=2.0


In [ ]:
import os
os.chdir('modelscope-agent/demo')


import sys
sys.path.append('../')

### 读取环境变量文件和工具config配置

In [ ]:
from dotenv import load_dotenv
import os
from modelscope.utils.config import Config

# 配置环境变量，里面需要配置modelscope token来remote调用AI模型
os.environ['TOOL_CONFIG_FILE'] ='../config/cfg_tool_template.json'
os.environ['OUTPUT_FILE_DIRECTORY'] = '/tmp'

# ModelScope Token从这里获取: https://modelscope.cn/my/myaccesstoken
print('请输入ModelScope Token，可以从这里获取: https://modelscope.cn/my/myaccesstoken')
os.environ['MODELSCOPE_API_TOKEN'] = input()

# 读取工具调用CONFIG文件，包括工具的名称和远程请求url
tool_cfg_file = os.getenv('TOOL_CONFIG_FILE')
print(tool_cfg_file)
tool_cfg = Config.from_file(tool_cfg_file)

### 构建工具调用向量检索

In [ ]:
from langchain.embeddings import ModelScopeEmbeddings, DashScopeEmbeddings
from langchain.vectorstores import FAISS
from modelscope_agent.retrieve import ToolRetrieval

# 第一种向量方案，基于阿里云DashScope向量API服务方式，需要开通DashScope文本向量服务()
# embeddings = DashScopeEmbeddings(model="text-embedding-v1")

# 第二种向量方案，基于魔搭社区开源向量模型
model_id = 'damo/nlp_corom_sentence-embedding_chinese-base'
embeddings = ModelScopeEmbeddings(model_id=model_id)

# 基于FAISS构建向量检索引擎
tool_retrieval = ToolRetrieval(embeddings, FAISS) 


### 中枢大模型启动

In [ ]:
from modelscope_agent.llm import LLMFactory
from modelscope.hub.api import HubApi

model_name = 'modelscope-agent-qwen-7b'
model_cfg = {
    'modelscope-agent-qwen-7b':{
        'model_id': 'damo/MSAgent-Qwen-7B',
        'model_revision': 'v1.0.2',
        'use_raw_generation_config': True,
        'custom_chat': True
    }
}


llm = LLMFactory.build_llm(model_name, model_cfg)


### Agent构建

In [ ]:
from modelscope_agent.prompt import MSPromptGenerator
from modelscope_agent.agent import AgentExecutor

prompt_generator = MSPromptGenerator(system_template="你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。\
你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。")

# 构建Agent，需要传入llm，工具配置config以及工具检索
agent = AgentExecutor(llm, tool_cfg, tool_retrieval=tool_retrieval, prompt_generator=prompt_generator)

In [ ]:
# 重置对话，清空对话历史
agent.reset()

# remote=True为调用modelscope api，该服务免费支持QPS较低，建议部署在本地，将remote=False
agent.run('使用地址识别模型，从下面的地址中找到省市区等元素，地址：浙江杭州市江干区九堡镇三村村一区', remote=False)

In [ ]:
# Multi-turn tool-use
agent.reset()
agent.run('写一个20字左右简短的小故事')
agent.run('用女声念出来', remote=False)